# Modifications de résultats des sondages

## Import préliminaires des librairies

In [1]:
import re
import numpy as np
import pandas as pd
import locale 
from locale import atof

# Read files
import PyPDF2
from pdfminer.high_level import extract_text

In [2]:
fp="rapport/2018-12-08.pdf"
text = extract_text(fp,page_numbers=[23])
question1="Êtes-vous  satisfait  ou  mécontent  d"
question2="Êtes-vous satisfait ou mécontent d"
# print(text.find(question1))
# print(text.find(question2))
# print(len(question2))
print(re.sub('\.\.\. (?![\n])','...\n',text))

Baromètre de l'action politique / 7 et 8 Décembre 2018

s37. Quel jugement portez-vous sur l'action de Christophe CASTANER?

BASE

Très
favorable

Plutôt
favorable

ST Favorable

Plutôt
défavorable

Très
défavorable

ST
Défavorable

Nsp

Evolution ST
Favorable

Evolution ST
Défavorable

ENSEMBLE ..................................................................

971

2,4

16,2

18,5

24,9

39,5

64,4

17,1

-1,0

14,1

Sexe

Age

Homme ..........................................................................
Femme ...........................................................................

462
509

18 à 24 ans ....................................................................
25 à 34 ans ....................................................................
35 à 44 ans ....................................................................
45 à 59 ans ...................................................................
60 à 69 ans ..........................................................

In [3]:
with open(fp,"rb") as pdf_file:
    pdf_reader= PyPDF2.PdfFileReader(pdf_file)
    isEncrypted = pdf_reader.getIsEncrypted()
    nbPages= pdf_reader.numPages

    # Define columns
    page_content=pdf_reader.getPage(15).extractText()
print(page_content)

BASE
A1.Jugementportésurl'actiond'EmmanuelMACRONentantqueprésidentdela
République
EvolutionST
Favorable
EvolutionST
Défavorable
Très
favorable
Plutôt
favorable
STFavorable
Plutôt
défavorable
Très
défavorable
ST
Défavorable
Nsp
ENSEMBLE..................................................................
971
3,2
16,4
19,6
26,2
49,8
76,0
4,4
-6,4
5,7
Sexe
Homme..........................................................................
462
3,0
16,3
19,2
31,9
46,3
78,3
2,5
-12,1
11,4
Femme...........................................................................
509
3,5
16,5
20,0
20,9
53,0
74,0
6,0
-1,2
0,5
Age
18à24ans....................................................................
103
12,0
18,5
30,5
26,8
32,5
59,3
10,2
-0,3
-5,4
25à34ans....................................................................
154
1,8
11,2
13,1
23,9
57,1
81,0
6,0
-10,1
9,5
35à44ans....................................................................
150
4,6
20,4
25,0
22,9
48,0
70,9
4,1
4,7
-6,2
45à59ans.......

## Extraction utilisant une lecture de PDF alternative

In [4]:
def fill_with_zeros(bases,groups,c_cat):
    while len(bases[c_cat])<len(groups[c_cat]):
        bases[c_cat].insert(0,0)

def clean_page(page_text):
    text=re.sub('\.\.\. (?![\n])','...\n',page_text)
    return text

def find_fill_subset(values,size):
    fitting_set=[]
    count=len(values[0])
    index_set=1
    for i in range(1,len(values)):
        if count == size:
            break
        else:
            count+=len(values[i])+1
            index_set+=1
        if count>size:
            count-=len(values[i])+1
            index_set-=1
            break
    if count>size:
        print('count size wrong. Count {} size {}'.format(count,size))
        print(values[:4])
        raise AssertionError()
    while count!=size:
        fitting_set.append(0)
        count+=1
    fitting_set.extend(values[0])
    for i in range(1,index_set):
        fitting_set.append(0)
        fitting_set.extend(values[i])
    return fitting_set, index_set
    
def groups_size(cat,groups):
    count=0
    for c in cat:
        count+=len(groups[c])
    return count

In [5]:
def extract_groups(page_text):
    text=page_text[page_text.find('18 à 24 ans'):]
    cat=['ENSEMBLE','Sexe','Age']
    groups={'ENSEMBLE':['ENSEMBLE'],'Sexe':['Homme','Femme'],'Age':[]}
    for line in text.split('\n'):
        if re.search('[a-zA-Z]', line) and line!='sd&p':
            is_group=re.search(' [\.]+', line)
            if is_group:
                group=line[:is_group.start()].rstrip()
                groups[cat[-1]].append(group)
            elif line=="ST Non expr.":
                line=line.replace(' ','')
                group=line
                groups[cat[-1]].append(group)
            else:
                line=line.replace(' ','')
                cat.append(line)
                groups[line]=[]
    return cat,groups

In [6]:
def extract_base(page_text,cat,groups):
    """Extract the base for all values
    page_text : raw text extracted
    cat : list of categories found
    groups : list of sub categories
    """
    ## Clean text and ignore part of the begining
    # Some lines with ... do not jump lines at the end
    # making it difficult to parse
    text=clean_page(page_text)
    text=text[text.find('ENSEMBLE'):]
    # iterable on categories
    i_cat=iter(cat)
    c_cat=next(i_cat)
    # bases will store the bases for the categories
    bases={'ENSEMBLE':[]}
    # base_found is true if for the current cat at least one base is found
    # if true no double line jump should be found before next cat
    base_found=False
    for line in text.split('\n'):
        line=line.replace(' ','')
        if not base_found:
            # search until first base is found, ignore everything else
            if re.search('^\d+$',line):
                bases[c_cat].append(int(line))
                base_found=True
        else:
            # no more double line jump, ignore text or float number
            if line!='':
                if re.search('^\d+$',line):
                    bases[c_cat].append(int(line))
            else:
                # Check if numbers of groups and bases match or add zero
                # Throw an alert in this case
                if len(bases[c_cat])<len(groups[c_cat]):
                    print("Size for {} do not match. Base size={}, groups size={}".format(c_cat,len(bases[c_cat]),len(groups[c_cat])))
                    print("Bases found {}".format(bases[c_cat]))
                    fill_with_zeros(bases,groups,c_cat)
                # reset everything and go to the next cat
                try:
                    c_cat=next(i_cat)
                    bases[c_cat]=[]
                    base_found=False
                except StopIteration:
                    return bases

In [7]:
def extract_ensemble(page_text):
    text=clean_page(page_text)
    text=text[text.find('ENSEMBLE'):]
    values=[]
    for line in text.split('\n')[1:]:
        if re.search('-?\d+,\d+',line):
            values.append(float(line.replace(',','.')))
        if re.search('[a-zA-Z]',line):
            return values

In [8]:
def get_all_values(page_text):
    text=clean_page(page_text)
    text=text[text.find('Sexe'):]

    values=[]
    sub_v=[]
    for line in text.split('\n'):
        if re.search('-?\d+,\d+',line):
            sub_v.append(float(line.replace(',','.')))
        elif line=='' and len(sub_v)!=0:
            values.append(sub_v)
            sub_v=[]
    return values

In [9]:
def fill_col(all_v,cat,groups,ens_val,col_names,df,ignore_val_ens):
    # ignore_val_ens ignore val in ens for last column if true
    v_index=0
    col_size=groups_size(cat,groups)
    for col_index in range(len(col_names)):
        col=[]
        for c_cat in cat:
            last_col_cond= col_index+1==len(col_names) and ignore_val_ens
            if c_cat=='ENSEMBLE' and not last_col_cond:
                col.append(ens_val[col_index])
            else:
                l_cat=len(groups[c_cat])
                f_set, new_index =find_fill_subset(all_v[v_index:],l_cat)
                v_index+=new_index
                col.extend(f_set)
                if c_cat=='ENSEMBLE' and last_col_cond:
                    ens_val.extend(f_set)
                    print('verif {}'.format(ens_val))
        if len(col)!=col_size:
            print("size {} expected {}".format(len(col),col_size))
            print(col)
        df[col_names[col_index]]=col
    return df



In [10]:
def extract_ipsos_page_alt(page_text,name,base,cat,groups):
    ens_val=extract_ensemble(page_text)

    # Assert ens_val is correct
    correct=''
    last_col_ignore=False
    if len(ens_val)<9:
        last_col_ignore=True
        # print('Incorrect size\n values: {}'.format(ens_val))
        # while correct=='' or len(ens_val)<9:
        #     correct=input('Do you want to correct it ?')
        #     ens_val=[float(x) for x in correct.split(',')]
        # print('thank you')
    # end assertion

    v=get_all_values(page_text)
    df=pd.DataFrame(columns=['Nom','Categorie','Groupe','Base'])
    for c_cat in cat:
        c_df=pd.DataFrame(columns=['Categorie','Groupe','Base'])
        c_df['Groupe']=groups[c_cat]
        c_df['Base']=base[c_cat]
        c_df['Categorie']=c_cat
        c_df['Nom']=name
        df=pd.concat([df,c_df],ignore_index=True)

    df=fill_col(v,cat,groups,ens_val,["Très favorable","Plutôt favorable","ST Favorable","Plutôt défavorable","Très défavorable","ST Défavorable","Nsp","Evolution ST Favorable","Evolution ST Défavorable"],df,last_col_ignore)
    return df

## Extraction utilisant une lecture de PDF

In [11]:
def extract_name(page_text):
    # Extract name
    question1="Jugementportésurl'actiond"
    question2="Queljugementportez-voussurl'actiond"
    fullname=""
    for l in page_text.split('\n'):
        if question1 in l:
            fullname=l[l.index(question1)+len(question1)+1:l.index("entantque")]
        if question2 in l:
            fullname=l[l.index(question2)+len(question2)+1:l.index("?")]
    for i in range(len(fullname)):
        if fullname[i:].isupper():
            prenom=fullname[:i]
            nom=fullname[i:]
            break
    return prenom,nom

In [32]:
def add_tuple(df,new_row,new_person):
    if new_person:
        new_row.append(0)
        new_row.append(0)
    # Not present value
    # Consider missing value is only TFavorable, tDefavorable or NSP
    if len(new_row)<10:
        existing_line=df[(df.Nom==new_row[0]) & (df.Categorie==new_row[1])  & (df.Groupe==new_row[2])]
        if len(existing_line!=0):
            alt_values=existing_line.iloc[0].to_list()[3:]
            new_row.extend(alt_values)
        else:
            #manual correction
            print(new_row)
            new_row=new_row[0:3]
            while len(new_row)<13:
                v=input('valeur ?')
                new_row.extend([float(x) for x in v.split(',')])
    while len(new_row)<13:
        all_favorable=np.sum(new_row[4:6])
        st_favorable=new_row[6]
        if np.abs(st_favorable-all_favorable)>=0.2:
            new_row.insert(4,0)
        else:
            all_defavorable=np.sum(new_row[7:9])
            st_defavorable=new_row[9]
            if np.abs(st_defavorable-all_defavorable)>=0.2:
                new_row.insert(7,0)
            else:
                new_row.insert(10,0)
        

    df=df.append(pd.DataFrame([new_row],columns=df.columns),ignore_index=True)
    return df
    

def extract_ipsos_page(df,page_text,columns):
    new_person="Evolution" not in page_text
    page_text=page_text.replace(",",".")
    page_text=page_text.replace("Moinsde1.200\n","Moinsde1.200€")
    page_text=page_text.replace("De1.200-2.000\n","De1.200-2.000€")
    page_text=page_text.replace("De2.000-3.000\n","De2.000-3.000€")
    page_text=page_text.replace("3.000\netplus","3.000€etplus")
    #Extract name
    prenom,nom=extract_name(page_text)
    fullname= prenom+' '+nom
    if fullname not in df.Nom.unique():
        print(fullname)
    # Filter text
        page_text=page_text[page_text.index("ENSEMBLE"):page_text.index("Baromètredel'actionpolitique")]


        number_regex="^-?[0-9]+(.[0-9]+)?$"
        categorie="ENSEMBLE"
        new_row=[]
        for l in page_text.split('\n'):
            # Test if number
            if re.match(number_regex,l):
                new_row.append(float(l))
            # Test if new groupe
            # It add the new tuple
            elif "...." in l:
                # Deal with ENSEMBLE categorie
                if len(new_row)>1:
                    df=add_tuple(df,new_row,new_person)  
                new_row=["{} {}".format(prenom,nom),categorie]
                new_row.append(l.strip('.'))
            else:
                categorie=l
        df=add_tuple(df,new_row,new_person)  
    return df


In [71]:
        
def extract_ipsos_data_from_pdf(file_path,page_range,df=None):
    with open(file_path,"rb") as pdf_file:
        pdf_reader= PyPDF2.PdfFileReader(pdf_file)
        isEncrypted = pdf_reader.getIsEncrypted()
        nbPages= pdf_reader.numPages

        # Define columns
        columns=["Nom","Categorie","Groupe","Base","Très favorable","Plutôt favorable","ST Favorable","Plutôt défavorable","Très défavorable","ST Défavorable","Nsp","Evolution ST Favorable","Evolution ST Défavorable"]
        if df is None:
            df=pd.DataFrame(columns=columns)
        for i in range(page_range[0],page_range[1]+1):
            page_content=pdf_reader.getPage(i).extractText()
            df=extract_ipsos_page(df,page_content,columns)
    return df


In [36]:


def extract_ipsos_alt(filepath,p_start,p_end):
    with open(filepath,"rb") as pdf_file:
        pdf_reader= PyPDF2.PdfFileReader(pdf_file)
        isEncrypted = pdf_reader.getIsEncrypted()
        nbPages= pdf_reader.numPages

        # Define columns
        columns=["Nom","Categorie","Groupe","Base","Très favorable","Plutôt favorable","ST Favorable","Plutôt défavorable","Très défavorable","ST Défavorable","Nsp","Evolution ST Favorable","Evolution ST Défavorable"]
        df=pd.DataFrame(columns=columns)
        page_text_pdfminer = extract_text(filepath,page_numbers=[p_start])
        cat,groups=extract_groups(page_text_pdfminer)
        base=extract_base(page_text_pdfminer,cat,groups)
        for i in range(p_start,p_end+1):
            page_text_pyPDF=pdf_reader.getPage(i).extractText()
            page_text_pdfminer = extract_text(filepath,page_numbers=[i])
            name=extract_name(page_text_pyPDF)
            name=name[0]+' '+name[1]
            print(name)
            try:
                c_df=extract_ipsos_page_alt(page_text_pdfminer,name,base,cat,groups)
                df=pd.concat([df,c_df],ignore_index=True)
            except AssertionError:
                print('{} has been skipped'.format(name))
    return df

In [38]:
filename="2018-12-08"
first_p=15
last_p=50
# df=extract_ipsos_alt("rapport/"+filename+".pdf",first_p,last_p)
df=extract_ipsos_data_from_pdf("rapport/"+filename+".pdf",(first_p,last_p),None)
df


Size for PCSdelapersonnederéférencedufoyer do not match. Base size=7, groups size=8
Bases found [45, 120, 153, 186, 122, 279, 66]
Emmanuel MACRON
count size wrong. Count 4 size 2
[[0.8, 0.3, 7.1, 11.4], [16.3, 16.5], [18.5, 11.2, 20.4, 13.3, 17.9, 20.4, 14.2, 15.8, 19.2], [39.4, 26.3, 13.3, 15.6, 4.2, 18.1, 7.8]]
Emmanuel MACRON has been skipped
Edouard PHILIPPE
verif [1.8, 19.9, 21.7, 32.9, 39.7, 72.6, 5.7, -5.0, 7.0]
Martine AUBRY
François BAROIN
verif [3.4, 22.6, 26.0, 28.6, 20.1, 48.6, 25.4, 3.2, 1.0]
François BAYROU
verif [2.1, 26.2, 28.3, 38.8, 22.1, 60.9, 10.7, 0.1, 3.5]
Xavier BERTRAND
Jean-Michel BLANQUER
verif [3.9, 17.4, 21.3, 24.0, 16.9, 41.0, 37.7, -1.7, 6.5]
Agnès BUZYN
Christophe CASTANER
Eric CIOTTI
Gérard COLLOMB
count size wrong. Count 9 size 8
[[11.1, 17.0, 18.5, 20.2, 21.0, 24.6, 14.6, 19.6, 22.9], [14.0, 21.7, 22.1, 9.6, 15.8, 24.6, 22.4], [17.0, 23.0, 22.2, 13.1, 13.7, 23.6, 19.4], [18.4, 14.3, 13.2, 27.0, 26.4, 19.4]]
Gérard COLLOMB has been skipped
Gérald DARMAN

,Nom,Categorie,Groupe,Base,Très favorable,Plutôt favorable,ST Favorable,Plutôt défavorable,Très défavorable,ST Défavorable,Nsp,Evolution ST Favorable,Evolution ST Défavorable
0,Edouard PHILIPPE,ENSEMBLE,ENSEMBLE,971,1.8,19.9,21.7,32.9,39.7,72.6,5.7,-5.0,7.0
1,Edouard PHILIPPE,Sexe,Homme,462,1.5,20.7,22.2,34.9,39.4,74.3,3.5,-9.6,9.3
2,Edouard PHILIPPE,Sexe,Femme,509,2.1,19.2,21.3,31.2,39.9,71.1,7.6,-0.9,4.9
3,Edouard PHILIPPE,Age,18 à 24 ans,103,2.0,24.7,26.7,33.2,27.9,61.1,12.2,4.2,1.1
4,Edouard PHILIPPE,Age,25 à 34 ans,154,1.9,11.1,13.0,33.8,43.6,77.4,9.6,-10.5,15.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,Laurent WAUQUIEZ,VotePrésidentielle20171ertour,ST Non expr.,238,1.5,9.6,11.0,24.2,31.8,55.9,33.0,5.2,1.6
2096,Laurent WAUQUIEZ,VotePrésidentielle20172ndtour,Emmanuel Macron,379,0.7,12.2,13.0,27.9,47.5,75.4,11.7,-1.6,2.4
2097,Laurent WAUQUIEZ,VotePrésidentielle20172ndtour,Marine Le Pen,194,3.9,20.0,23.9,33.4,23.8,57.2,18.8,-3.3,5.8
2098,Laurent WAUQUIEZ,VotePrésidentielle20172ndtour,ST Expr.,573,1.8,14.9,16.7,29.8,39.5,69.2,14.1,-2.2,3.6


In [73]:
df.Nom.unique()

array(['Edouard PHILIPPE', 'Martine AUBRY', 'François BAROIN',
       'François BAYROU', 'Xavier BERTRAND', 'Jean-Michel BLANQUER',
       'Agnès BUZYN', 'Christophe CASTANER', 'Eric CIOTTI',
       'Gérald DARMANIN', 'Nicolas DUPONT-AIGNAN', 'Christian ESTROSI',
       'Olivier FAURE', 'François FILLON', 'Benjamin GRIVEAUX',
       'Benoît HAMON', 'Anne HIDALGO', 'François HOLLANDE',
       'Nicolas HULOT', 'Alain JUPPE', 'Jean-Yves LEDRIAN',
       'Bruno LEMAIRE', 'Marine LEPEN', 'Marion MARECHAL-LEPEN',
       'Valérie PECRESSE', 'Muriel PENICAUD', 'Ségolène ROYAL',
       'Nicolas SARKOZY', 'Marlène SCHIAPPA', 'Laurent WAUQUIEZ',
       'Emmanuel MACRON', 'Gérard COLLOMB', 'François DERUGY',
       'Jack LANG', 'Jean-Luc MELENCHON', 'Manuel VALLS'], dtype=object)

In [16]:
# for n in df_alt.Nom.unique():
#     t1=df[df.Nom==n].iloc[0]
#     t2=df_alt[df_alt.Nom==n].iloc[0]
#     if t1!=t2:
#         print('PB:\n{}\n{}'.format(t1,t2))

In [17]:
# df3=pd.concat([df,df2],ignore_index=True)
# df3.to_pickle('data/'+filename+'.p')

In [44]:
# Reading IPSOS report

# with open("rapport/2020-10-14.pdf","rb") as pdf_file:
#     pdf_report= PyPDF2.PdfFileReader(pdf_file)
#     isEncrypted = pdf_report.getIsEncrypted()
#     nbPages= pdf_report.numPages

#     page1=pdf_report.getPage(15)
#     text=page1.extractText().replace(',','.')
#     text = extract_text(pdf_file,page_numbers=[14])

# print(text)

print(df.isnull().values.any())
df.head(10)
df.describe()
df[df.Base==0]

False


30

## Find mistakes and try to correct them

In [40]:
def find_fav_mistake(df):
    tfav=df['Très favorable']
    pfav=df['Plutôt favorable']
    stfav=df['ST Favorable']
    fav_df= df[np.abs(tfav+pfav-stfav)>0.11]
    return fav_df

def find_def_mistake(df):
    tdef=df['Très défavorable']
    pdef=df['Plutôt défavorable']
    stdef=df['ST Défavorable']
    def_df= df[np.abs(tdef+pdef-stdef)>0.11]
    return def_df

def find_nsp_mistake(df):
    stfav=df['ST Favorable']
    stdef=df['ST Défavorable']
    nsp=df['Nsp']
    tot_df= df[np.abs(stfav+stdef+nsp-100)>0.11]
    return tot_df

def find_mistake(df):
    fav_df= find_fav_mistake(df)
    def_df= find_def_mistake(df)
    tot_df=find_nsp_mistake(df)
    error_df=pd.concat([fav_df,def_df,tot_df])
    return error_df

In [59]:
def realign_fav(df,i=0,imax=-1):
    df_err=find_fav_mistake(df)
    zeros_cols= [x for x in ["Très favorable","Plutôt favorable","ST Favorable"] if df[x].all()==0]
    if imax==-1:
        imax=len(zeros_cols)*len(df)
    if not df_err.empty and i<=imax and len(zeros_cols)>0:
        col=zeros_cols[0]
        values=df_err[col].tolist()
        values=values[1:]+[values[0]]
        df.loc[df_err.index,col]=values
        df=realign_fav(df,i+1)
    return df

def realign_def(df,i=0,imax=-1):
    df_err=find_fav_mistake(df)
    zeros_cols= [x for x in ["Très défavorable","Plutôt défavorable","ST Défavorable"] if df[x].all()==0]
    if imax==-1:
        imax=len(zeros_cols)*len(df)
    if not df_err.empty and i<=imax:
        col=zeros_cols[0]
        values=df_err[col].tolist()
        values=values[1:]+[values[0]]
        df.loc[df_err.index,col]=values
        df=realign_def(df,i+1)
    return df

def realign_nsp(df,i=0,imax=-1):
    df_err=find_fav_mistake(df)
    zeros_cols= [x for x in ["ST Favorable","ST Défavorable","Nsp"] if df[x].all()==0]
    if imax==-1:
        imax=len(zeros_cols)*len(df)
    if not df_err.empty and i<=imax:
        col=zeros_cols[0]
        values=df_err[col].tolist()
        values=values[1:]+[values[0]]
        df.loc[df_err.index,col]=values
        df=realign_nsp(df,i+1)
    return df

    



In [56]:
def correct_mistakes(df):
    df_err=find_fav_mistake(df)
    for nom in df_err.Nom.unique():
        for cat in df_err[df_err.Nom==nom].Categorie.unique():
            cond=((df_err.Nom==nom) & (df_err.Categorie==cat))
            df_err.loc[cond]=realign_fav(df_err[cond])
    df.loc[df_err.index]=df_err

    df_err=find_def_mistake(df)
    for nom in df_err.Nom.unique():
        for cat in df_err[df_err.Nom==nom].Categorie.unique():
            cond=((df_err.Nom==nom) & (df_err.Categorie==cat))
            df_err.loc[cond]=realign_def(df_err[cond])
    df.loc[df_err.index]=df_err

    df_err=find_nsp_mistake(df)
    for nom in df_err.Nom.unique():
        for cat in df_err[df_err.Nom==nom].Categorie.unique():
            cond=((df_err.Nom==nom) & (df_err.Categorie==cat))
            df_err.loc[cond]=realign_nsp(df_err[cond])
    df.loc[df_err.index]=df_err
    return df

In [76]:
df_err=find_mistake(df)
df_err
# df=correct_mistakes(df)


,Nom,Categorie,Groupe,Base,Très favorable,Plutôt favorable,ST Favorable,Plutôt défavorable,Très défavorable,ST Défavorable,Nsp,Evolution ST Favorable,Evolution ST Défavorable
188,François BAROIN,Proximitépolitique,EELV,66,3.5,11.9,11.9,39.0,21.6,60.6,27.5,2.0,11.3
194,François BAROIN,Proximitépolitique,RN,122,0.7,20.7,21.6,26.6,24.4,51.0,27.4,10.4,-4.2
200,François BAROIN,VotePrésidentielle20171ertour,B. Hamon,47,0.0,13.3,16.7,34.4,26.5,61.0,22.3,0.4,1.9
257,François BAYROU,Proximitépolitique,PS,101,7.5,37.2,39.2,37.4,16.3,53.6,7.1,-2.3,5.4
258,François BAYROU,Proximitépolitique,EELV,66,0.9,37.6,37.6,36.3,14.3,50.6,11.7,-6.2,5.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2252,François DERUGY,PCSdelapersonnederéférencedufoyer,Agriculteurexploitant,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2322,Jack LANG,PCSdelapersonnederéférencedufoyer,Agriculteurexploitant,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2392,Jean-Luc MELENCHON,PCSdelapersonnederéférencedufoyer,Agriculteurexploitant,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2442,Jean-Luc MELENCHON,VotePrésidentielle20171ertour,F.Fillon,147,8.3,8.3,16.6,0.0,74.7,91.3,0.4,3.8,-3.0


In [77]:
df_err=find_mistake(df)
while len(df_err)>0:
    index=df_err.iloc[0:1].index[0]
    val=df_err.iloc[0].values[3:]
    print(df_err.iloc[0])
    print(val)
    correct=[]
    cols=["Base","Très favorable","Plutôt favorable","ST Favorable","Plutôt défavorable","Très défavorable","ST Défavorable","Nsp","Evolution ST Favorable","Evolution ST Défavorable"]
    while len(correct)!=10:
        correct=input()
        correct=[float(x) for x in correct.split(' ')]
    print(correct)
    df.loc[index,cols]=correct
    df_err=find_mistake(df)

Nom                            François BAROIN
Categorie                   Proximitépolitique
Groupe                                    EELV
Base                                        66
Très favorable                             3.5
Plutôt favorable                          11.9
ST Favorable                              11.9
Plutôt défavorable                          39
Très défavorable                          21.6
ST Défavorable                            60.6
Nsp                                       27.5
Evolution ST Favorable                       2
Evolution ST Défavorable                  11.3
Name: 188, dtype: object
[66 3.5 11.9 11.9 39.0 21.6 60.6 27.5 2.0 11.3]


ValueError: could not convert string to float: 

In [78]:

# df.to_pickle('data/'+filename+'.p')

### Uncomment to try correct some flaws

In [28]:
# df_alt=extract_ipsos_alt("rapport/"+filename+".pdf",13,54)
# correct_mistakes(df_alt)
# find_mistake(df_alt)
def_err_alt=find_mistake(df_alt)
for i in df_err.index:
val=df.loc[i]
alt_val=df_alt[(df_alt.Nom==val.Nom) & (df_alt.Categorie==val.Categorie) & (df_alt.Groupe==val.Groupe)]
if not alt_val.empty and alt_val.index[0] not in def_err_alt.index:
    df.loc[i]=alt_val.iloc[0]
find_mistake(df)

IndentationError: expected an indented block (<ipython-input-28-6c4cc12a0387>, line 6)